In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pytrends

In [3]:
! pip install pycountry
#convert country names to ISO-3 codes

  Using cached pycountry-24.6.1-py3-none-any.whl.metadata (12 kB)
Using cached pycountry-24.6.1-py3-none-any.whl (6.3 MB)


Step 1 - Imports

In [5]:
import time
import pycountry
import matplotlib.pyplot as plt
import plotly.express as px
from pytrends.request import TrendReq
from pytrends.exceptions import TooManyRequestsError

%matplotlib inline  


Step 2: Safe request function

In [6]:
def safe_pytrends_request(func, retries=5, wait=60):
    """
    Safely call a pytrends function with automatic retry on 429 errors.
    """
    for attempt in range(retries):
        try:
            df = func()
            if df.empty:
                print("Warning: returned DataFrame is empty.")
            return df
        except TooManyRequestsError:
            print(f"429 error detected → waiting {wait} seconds...")
            time.sleep(wait)
            wait *= 2  # exponential backoff
    raise Exception("Too many retries, Google keeps blocking requests")


Step 3: Initialize pytrends

In [7]:
# Initialize pytrends
pytrends = TrendReq(hl='en-US', tz=360)

# List your keywords
keywords = ["cloud computing", "AI", "data science"]

# Build payload (one at a time or up to 5 keywords)
pytrends.build_payload(keywords, timeframe='today 12-m')


Step 4: Fetch interest over time

In [ ]:
time_df = safe_pytrends_request(pytrends.interest_over_time)
print(time_df.head())


429 error detected → waiting 60 seconds...
429 error detected → waiting 120 seconds...
429 error detected → waiting 240 seconds...


Step 5: Plot time series for each keyword

In [ ]:
plt.figure(figsize=(12,6))

for kw in keywords:
    if kw in time_df.columns:
        plt.plot(time_df.index, time_df[kw], marker='o', label=kw)

plt.title("Search Interest Over Time")
plt.xlabel("Date")
plt.ylabel("Interest")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


Step 6: Fetch interest by region

In [ ]:
region_df = safe_pytrends_request(pytrends.interest_by_region)
region_df = region_df.reset_index(names='geoName')
print(region_df.head())


Step 7: Convert country names to ISO-3 codes

In [ ]:
def country_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

region_df['iso_alpha'] = region_df['geoName'].apply(country_to_iso3)
region_df = region_df.dropna(subset=['iso_alpha']).copy()


Step 8: Plot interactive choropleth map

In [ ]:
# Example: plot first keyword
keyword = keywords[0]

fig = px.choropleth(
    region_df,
    locations='iso_alpha',
    locationmode='ISO-3',
    color=keyword,
    hover_name='geoName',
    title=f"Search Interest for '{keyword}' by Country",
    color_continuous_scale='Blues'
)
fig.show()
